# Sentinel-2 Super-Resolution (4×)

**Upscale Sentinel-2 imagery from 10m/pixel → 2.5m/pixel using SwinIR**

- ✅ **Streaming Inference**: Fetches live data from GEE (No massive downloads)
- ✅ **SwinIR Model**: Pretrained deep swin transformer for restoration
- ✅ **Hallucination Guardrails**: Ensures geospatial consistency

Enable GPU: `Runtime → Change runtime type → T4 GPU`

In [ ]:
# 1. Clone repository & install
!git clone https://github.com/Rishikarnatakam/Klymo.git
%cd Klymo
!pip install -q -r requirements.txt

In [ ]:
# 2. Verify GPU
import torch
assert torch.cuda.is_available(), "GPU not available! Enable it in Runtime settings."
print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
DEVICE = 'cuda'

In [ ]:
# 3. GEE Authentication (for live tile fetching)
from google.colab import auth
auth.authenticate_user()

import ee
# Authenticate to Initialize
ee.Authenticate()
ee.Initialize(project='klymo-486313')
print("✓ GEE authenticated")

In [ ]:
# 4. Load Pretrained SwinIR Model
from src.models.swinir import load_swinir_model

print("Loading pretrained SwinIR (Generic 4× SR)...")
model = load_swinir_model(device=DEVICE)
print("✓ Model loaded (Pretrained on DIV2K/Flickr2K)")

In [ ]:
# 5. Fetch real Sentinel-2 tile (Streamed from GEE)
from src.data.gee_fetcher import GEEFetcher

fetcher = GEEFetcher(authenticate=False)
fetcher.authenticated = True
fetcher.ee = ee

# Stream tile from Delhi, India
print("Streaming Sentinel-2 tile from GEE...")
delhi_tile = fetcher.fetch_tile('delhi', tile_size=256)
print(f"✓ Streamed tile: {delhi_tile.shape} (No local storage)")

import matplotlib.pyplot as plt
plt.figure(figsize=(6, 6))
plt.imshow(delhi_tile)
plt.title('Sentinel-2 Delhi (10m/pixel) - Streamed from GEE')
plt.axis('off')
plt.show()

In [ ]:
# 6. Run Super-Resolution Pipeline
from src.inference.pipeline import SuperResolutionPipeline

pipeline = SuperResolutionPipeline(device=DEVICE)
results = pipeline.run(delhi_tile)

print(f"Input:  {results['lr'].shape} → 10m/pixel")
print(f"Output: {results['sr'].shape} → 2.5m/pixel (4× enhancement)")

In [ ]:
# 7. Visualize Results
import matplotlib.pyplot as plt
from src.data.preprocessing import to_8bit_visualization
from skimage.transform import resize

lr = results['lr']
bicubic = results['bicubic']
sr = results['sr']

lr_up = resize(lr, sr.shape[:2], order=0, preserve_range=True)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(to_8bit_visualization(lr_up))
axes[0].set_title('Low Resolution (10m/pixel)', fontsize=14)
axes[0].axis('off')

axes[1].imshow(to_8bit_visualization(bicubic))
axes[1].set_title('Bicubic 4× (Baseline)', fontsize=14)
axes[1].axis('off')

axes[2].imshow(to_8bit_visualization(sr))
axes[2].set_title('SwinIR 4× (2.5m/pixel)', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.savefig('outputs/visualizations/comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved comparison to outputs/visualizations/comparison.png")

In [ ]:
# 8. Quality Metrics & Hallucination Checks
from src.metrics.psnr import compute_psnr
from src.metrics.ssim import compute_ssim

psnr = compute_psnr(bicubic, sr)
ssim = compute_ssim(bicubic, sr)

print("="*40)
print("Quality Metrics (SwinIR vs Bicubic)")
print("="*40)
print(f"  PSNR: {psnr:.2f} dB")
print(f"  SSIM: {ssim:.4f}")
print("="*40)

if 'checks' in results:
    print("\nHallucination Checks:")
    for name, check in results['checks']['checks'].items():
        status = "✓" if check['passed'] else "✗"
        print(f"  {status} {name}: {check['score']:.4f}")

In [ ]:
# 9. Download Result
from google.colab import files
files.download('outputs/visualizations/comparison.png')
